In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import baikalfunctions as bfunc
import scheme_mar2023 as scheme
import mysecure

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.__version__

In [ ]:
# pd.set_option('display.min_rows', 100)
%matplotlib inline
plt.style.use('dark_background')   # try another styles: 'classic'
plt.rcParams['figure.figsize'] = [15, 5]
#plt.subplots_adjust(top=1, left=0, right=1, bottom=0)

In [ ]:
pwd = 'G:\\1_Data1\\98_BaikalJul2024Sea\\'
saveImgPath = pwd
saveDataPath = pwd

In [ ]:
obris = pd.read_csv("G:/1_Data1/obris/obris.dat",
                    header=0,
                    na_values='--',
                    sep='\t',
                    decimal=',',
                    )
obris.head(3)

### Concat all dataframes

### refactor table. remove unused columns, rename columns

In [ ]:
df = pd.read_csv(pwd+'BaikalJul24Sea.dat',
                 sep='\t',
                 skiprows=[1],
                 parse_dates=['DateTime'],
                 dayfirst=True,
                 na_values='--',
                 decimal=',',
                )

In [ ]:
df.describe()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
plt.scatter('long', 'lat', data=obris, s=1)
plt.scatter('Longitude', 'Latitude', data=df, s=0.2)

In [ ]:
df.shape, df.columns

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(7, 1)

axs[0].set_title('vCO2, ppm')
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver')

axs[1].set_title('vCH4, ppm')
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].set_ylim(1.5, 2.5)

axs[2].set_title('Temp')
axs[2].scatter(df.DateTime, df['TempEqu1'],s=1)
axs[2].scatter(df.DateTime, df['TempEqu2'],s=1)
axs[2].scatter(df.DateTime, df['TempAir'],s=2,c='red')
#axs[2].set_ylim(-2, 10)

axs[3].set_title('WaterFlow')
axs[3].scatter(df.DateTime, df['WaterFlowEqu1'], s=1)
axs[3].scatter(df.DateTime, df['WaterFlowEqu2'], s=1)

axs[4].set_title('PressAir')
axs[4].scatter(df.DateTime, df['PressAir'], s=1, c='red')
axs[5].set_title('Solar')
axs[5].scatter(df.DateTime, df['LightLX'])
axs[6].set_title('SolarUV')
axs[6].scatter(df.DateTime, df['LightUV'])



### Correction data  
 ** Be carefull !!!  Make it ONCE !!! **


In [ ]:
# gether/apply specific variables
### data corrections
#df['vCO2'] = df['vCO2'] * scheme.CO2PicarroScaut_cor[0] + scheme.CO2PicarroScaut_cor[1]
#df['vCH4'] = df['vCH4'] * scheme.CH4PicarroScaut_cor[0] + scheme.CH4PicarroScaut_cor[1]
# df['vCO2'] = df['CO2_dry']
# df['vCH4'] = df['CH4_dry']
df['LightLX'] = df['LightLX'] * scheme.solar_cor[0] + scheme.solar_cor[1]
df['LightUV'] = df['LightUV'] * scheme.solar_uv_cor[0] + scheme.solar_uv_cor[1]
df['PressAir'] = bfunc.pressConvert(df['PressAir'])
df['PressAir'] = df['PressAir'] * scheme.press_cor[0] + scheme.press_cor[1]

### servise data corrections
df['TempEqu1'] = df['TempEqu1'] * scheme.tempEqu1_cor[0] + scheme.tempEqu1_cor[1]
df['TempEqu2'] = df['TempEqu2'] * scheme.tempEqu2_cor[0] + scheme.tempEqu2_cor[1]

df['AirFlow'] = df['AirFlow'] * scheme.airflow_cor[0] + scheme.airflow_cor[1]
df['WaterFlowEqu1'] = df['WaterFlowEqu1'] * scheme.waterflowEqu1_cor[0] + scheme.waterflowEqu1_cor[1]
df['WaterFlowEqu2'] = df['WaterFlowEqu2'] * scheme.waterflowEqu2_cor[0] + scheme.waterflowEqu2_cor[1]


###  Remove bad data   
List of accidental cases to filter data (see notes)

In [ ]:
filter_list = [{'date_start': '19.02.2022 13:00', 'date_stop': '19.02.2022 13:32', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.nan},   # Picarro is not ready
               {'date_start': '20.02.2022 17:53', 'date_stop': '20.02.2022 17:55', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.nan},   # Picarro mulfunction
               {'date_start': '21.02.2022 07:14', 'date_stop': '21.02.2022 11:52', 'cols': ['vCO2', 'vCH4'], 'fill_with': np.nan},   # Picarro mulfunction
               {'date_start': '20.02.2022 18:14', 'date_stop': '21.02.2022 02:30', 'cols': ['vCH4'], 'fill_with': np.nan},   # Picarro mulfunction
              ]

for cycle in filter_list:
    for col in cycle['cols']:
        date_start = pd.to_datetime(cycle['date_start'], dayfirst=True)
        date_stop = pd.to_datetime(cycle['date_stop'], dayfirst=True)
        df.loc[(df['DateTime'] > date_start) & (df['DateTime'] < date_stop), col] = cycle['fill_with']

In [ ]:
cols = ['FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu']
for col in cols:
    df.loc[df[col] == 0, col] = np.nan
cols = ['WaterFlowEqu1', 'WaterFlowEqu2', 'AirFlow']
for col in cols:
    df.loc[df[col] < 0, col] = np.nan

### prepare data

In [ ]:
chnl = 'Channel'
if not chnl in df.columns:
    df[chnl] = 0
    for k, val in scheme.CHANNEL_COLS.items():
        df.loc[df[val] == 1, chnl] = k

In [ ]:
v_state_list = list(scheme.CHANNEL_COLS.values())
df.drop(v_state_list, inplace=True, errors='ignore')

#### split data on channel events
separate `CH4air` from whole massive. take minimum from each cycle

In [ ]:
df['event'] = 0
df.loc[df[chnl] != df[chnl].shift(1), ['event']] = 1
df['event'] = df['event'].cumsum()
df

In [ ]:
dfr = df.groupby(by=['event']).min()
dfr = dfr.loc[dfr[chnl] == 6,['DateTime', 'vCH4']]
dfr.rename({'vCH4': 'vCH4air'}, axis='columns', errors='ignore', inplace=True)
dfr

In [ ]:
df.drop('vCH4air', axis='columns', inplace=True, errors='ignore' )
df = pd.merge(left=df, right=dfr, left_on='DateTime', right_on='DateTime', how='left', suffixes=(None, 'air1'))
df.describe()

In [ ]:
ch_v = 'valid'
df[ch_v] = 1
df.loc[(df['event'] != df['event'].shift(-2)), ['valid']] = 0
df.loc[(df['event'] != df['event'].shift(8)), ['valid']] = 0

In [ ]:
df['vCO2air'] = df['vCO2'][(df[chnl] == 6) & (df[ch_v] == 1)]
#df['vCH4air'] = df['vCH4'][(df[chnl] == 6) & (df[ch_v] == 1)]
#df['vCH4air'] = df['vCH4air']  ## take minimal for each cycle

In [ ]:
df.index = df['DateTime']
df['vCO2air'] = df['vCO2air'].interpolate(method='time')
df['vCH4air'] = df['vCH4air'].interpolate(method='time')  ## `time` method mb better

In [ ]:
for col in ['vCO2air', 'vCH4air']:
    df[col] = df[col].rolling('10min').mean()

In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
plt.rcParams['figure.figsize'] = (15,5)
plt.scatter('DateTime', 'vCO2air', data=df,s=1, c='red', alpha=1)
plt.scatter('DateTime', 'vCO2', data=df, s=0.1, c='cyan', alpha=0.2)
#plt.ylim(1.8, 5)
#plt.xlim(pd.to_datetime('2024-07-27 00:00'), pd.to_datetime('2024-07-27 12:00'))


In [ ]:
plt.rcParams['figure.figsize'] = (15,5)
plt.scatter('DateTime', 'vCH4air', data=df,s=1, c='red', alpha=1)
plt.scatter('DateTime', 'vCH4', data=df, s=0.1, c='cyan', alpha=0.2)
plt.scatter('DateTime', 'vCH4air_med', data=df, s=0.1, c='green', alpha=0.2)
plt.ylim(1.8, 3)
plt.xlim(pd.to_datetime('2024-07-27 18:00'), pd.to_datetime('2024-07-28 10:00'))

In [ ]:
df['vCH4air_med'] = df['vCH4air'].median()

In [ ]:
df['pCO2air'] = df['vCO2air'] * df['PressAir']  # mkatm
df['pCH4air'] = df['vCH4air'] * df['PressAir']  # mkatm

In [ ]:
n = df['vCH4air'].describe()['25%']
n

In [ ]:
df.sample(20)

In [ ]:
df

In [ ]:
df = df.resample('20s', on='DateTime').mean()

In [ ]:
df.interpolate('time', inplace=True,)
df

In [ ]:
df.reset_index(drop=False, inplace=True)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, axs = plt.subplots(2, 1)

axs[0].set_title('CO2 air, ppm')
axs[0].set_ylim(400, 500)
axs[0].plot(df.DateTime, df['vCO2'], '-', c='silver', alpha=0.2)
axs[0].plot(df.DateTime, df['vCO2air'], 'r-', alpha=0.5)

axs[1].set_title('CH4 air, ppm')
axs[1].set_ylim(1.5, 3)
# axs[1].set_xlim(pd.to_datetime('2022-02-25 12:00'), pd.to_datetime('2022-02-25 14:00'))
axs[1].plot(df.DateTime, df['vCH4'], '-', c='silver')
axs[1].plot(df.DateTime, df['vCH4air'], 'b-')
fig.savefig(saveImgPath+'cAir_vs_time.png', transparent=False)

In [ ]:
df.columns

## RECOVERY !!!

In [ ]:
df['DateSec'] = df['DateTime'].astype('int64')//10**9
df['dTSec'] = df['DateSec'] - df['DateSec'].shift(1)
dt = df['dTSec'] / 60   ## delta time, min
ch_v = 'valid'
df[ch_v] = 1
for deep in range(1, 7):
    df.loc[df[chnl] != df[chnl].shift(deep), ch_v] = 0

#### recovery `CO2/CH4`, channel `1` (water in Jun 2024)

In [ ]:
equ_vol = scheme.equ_seatube_param['equ_vol']  # equivalent equ volume, l
equ_cap = scheme.equ_seatube_param['equ_cap']  # equilibrator capacity
wtr_flow_min = scheme.equ_seatube_param['water_flow_min'] 

In [ ]:
t_wtr = df['TempEqu1']
t_air = t_wtr
wtr_flow = df['WaterFlowEqu1']
air_flow = df['AirFlow']

In [ ]:
t_wtr.describe()

In [ ]:
### CO2 water
df['vCO2equ'] = df['vCO2'][(df[chnl] == 1) & (df[ch_v] == 1) & (wtr_flow > wtr_flow_min)]
solubility = bfunc.getSolubility(t_wtr, 'CO2')

density = bfunc.getDensity(t_air, df['PressAir'], 'CO2')
cGasAir = df['pCO2air'] / 1000000 * density
pGasEquAir = df['vCO2equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)

df['cCO2wtr'] = cGasWtr * 1000     # mg/l
df['pCO2wtr'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
### CH4 water
df['vCH4equ'] = df['vCH4'][(df[chnl] == 1) & (df[ch_v] == 1)]
solubility = bfunc.getSolubility(t_wtr, 'CH4')

density = bfunc.getDensity(t_air, df['PressAir'], 'CH4')
cGasAir = df['pCH4air'] / 1000000 * density
pGasEquAir = df['vCH4equ'] * df['PressAir']
cGasEquAir = pGasEquAir * density / 1000000  # g/l

tau = equ_vol/(air_flow+wtr_flow*equ_cap*solubility/density)
eternal = (cGasEquAir-cGasEquAir.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
cGasWtr = (eternal*(wtr_flow*equ_cap*solubility/density+air_flow)-air_flow*cGasAir)/(wtr_flow*equ_cap)
df['cCH4wtr'] = cGasWtr * 1000000000    # ng/l
df['pCH4wtr'] = cGasWtr * 1000000 / solubility  # mkatm

In [ ]:
df['pCH4wtr'].describe()

In [ ]:
df['pCO2'] = df['vCO2'] * df['PressAir']

In [ ]:
dfplot = df.loc[15100:15500,:]
plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
ax.set_title('CO2 water, mkatm')
#ax.set_xlim(pd.to_datetime('26.07.2024 23:00:00', dayfirst=True), pd.to_datetime('28.07.2024 02:00:00', dayfirst=True))
#ax.set_ylim(0, 600)
ax.scatter('DateTime', 'pCO2', data=dfplot, c='silver', s=2,  alpha=0.2)
ax.scatter('DateTime', 'pCO2wtr', data=dfplot, c='green', s=2)
ax.scatter('DateTime', 'pCO2air', data=dfplot, c='cyan', s=2)
fig.savefig(saveImgPath+'pCO2wtr_vs_time.png')

In [ ]:
col_name = 'pCO2wtr'
df['dif'] = df[col_name] - df[col_name].shift(-1)
df.loc[df['dif'] > 50, ['dif', 'pCO2wtr']]
#df.loc[320:340, ['dif', 'pCO2wtr']]

In [ ]:
## rolling average is applied inplace. Be cafelly, run this cell **ONCE**

for col in ['cCO2wtr', 'pCO2wtr', 'cCH4wtr', 'pCH4wtr']:
    df.loc[df[col] == np.inf , col] = np.nan 
    df.loc[df[col] == -np.inf , col] = np.nan
    df.loc[df[col] <= 0 , col] = np.nan
    df[col] = df[col].rolling(3, center=True).mean()

In [ ]:
count_recent = 30000
df['pCH4'] = df['vCH4'] * df['PressAir']
plt.rcParams['figure.figsize'] = [18, 5]
fig, ax = plt.subplots()
ax.set_title('CH4 water, mkatm')
#ax.set_xlim(pd.to_datetime('06.06.2024 10:00:00', dayfirst=True), pd.to_datetime('06.06.2024 19:00:00', dayfirst=True))
#ax.set_ylim(0, 20)
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4'], '-', c='silver')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4wtr'], 'r.')
ax.plot(df.tail(count_recent).DateTime, df.tail(count_recent)['pCH4air'], 'c.')
fig.savefig(saveImgPath+'pCH4wtr_vs_time.png')

In [ ]:
df.columns

In [ ]:
df['hour'] = df['DateTime'].dt.hour # + df['DateTime'].dt.minute / 60
df['hour']

In [ ]:
cols = ['DateTime', 'hour', 'DateSec', 'TempAir', 'PressAir', 'Precipitation', 'LightLX', 'LightUV',
        'Latitude', 'Longitude',
        'vCO2', 'vCH4', 'vH2O', 'AirFlow', 'Channel',
        'pCO2', 'pCH4',
        #'FluoNxRed', 'FluoNxGrn', 'FluoNxBlu', 'FluoKfaRed', 'FluoKfaGrn', 'FluoKfaBlu',
        'WaterFlowEqu1', 'WaterFlowEqu2', 'WaterFlowEqu3', 'WaterFlowEqu4',
        'TempEqu1', 'TempEqu2', 'TempEqu3',
        # 'cCO2chm', 'cCH4chm', 'CO2flux', 'CH4flux',
        'vCO2air', 'vCH4air', 'pCO2air', 'pCH4air',
        'cCO2wtr', 'pCO2wtr',
        'cCH4wtr', 'pCH4wtr',
       ]
# df = df.loc[:, cols]

In [ ]:
df.to_csv(f'{pwd}BaikalJul24Sea.dat',
          # columns=cols,
          sep='\t',
         )

In [ ]:
func_list = ['mean', 'std']
hourly = df.resample('1H', on='DateTime').agg(func_list)
hourly.to_csv(f'{pwd}BaikalJun2024hourly.dat',
              # columns=cols,
              sep='\t',
             )

In [ ]:
df.info()

## Station Statistics

In [ ]:
import mytools
geo = mytools.geo_gen_stations()
geo

In [ ]:
df['Station'] = ''

In [ ]:
df['Station'] = ''
DLAT = 0.005
DLNG = 0.01

for k, v in geo.items():
    df.loc[(df['Latitude'].between(v[0]-DLAT, v[0]+DLAT) & df['Longitude'].between(v[1]-DLNG, v[1]+DLNG)), 'Station'] = k

In [ ]:
dfg = df.groupby(by='Station')['cCH4wtr', 'cCO2wtr', 'Latitude', 'Longitude'].agg(['count', 'mean', 'median', 'std'])
dfg

In [ ]:
dfg.to_csv(pwd+'stationsCH4CO2.dat',
          sep='\t',
         )